In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Masking, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, regularizers, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
raw_data_prep_dir = '../raw_data/pads-parkinsons-disease-smartwatch-dataset-1.0.0/preprocessed/'

In [3]:
file_list = pd.read_csv(raw_data_prep_dir  + "file_list.csv")
file_list

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/pads-parkinsons-disease-smartwatch-dataset-1.0.0/preprocessed/file_list.csv'

In [ ]:
n_obs = len(file_list['id'])
n_obs

469

In [ ]:
time_data = []
y = []
for idx, subject_idx in enumerate(file_list['id']):
    file_name = raw_data_prep_dir + f'movement/{subject_idx:03d}_ml.bin'
    time_idx_data = np.fromfile(file_name, dtype=np.float32).reshape((-1, 976))
    time_data.append(time_idx_data)
    y_subject_id = file_list.iloc[idx]['label']
    y.append(y_subject_id)

In [ ]:
y

[0,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 0,
 1,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,


In [ ]:
channels = []
for task in ["Relaxed1", "Relaxed2", "RelaxedTask1", "RelaxedTask2", "StretchHold", "HoldWeight",
                "DrinkGlas", "CrossArms", "TouchNose", "Entrainment1", "Entrainment2"]:
    for device_location in ["LeftWrist", "RightWrist"]:
        for sensor in ["Acceleration", "Rotation"]:
            for axis in ["X", "Y", "Z"]:
                channel = f"{task}_{sensor}_{device_location}_{axis}"
                channels.append(channel)
channels

['Relaxed1_Acceleration_LeftWrist_X',
 'Relaxed1_Acceleration_LeftWrist_Y',
 'Relaxed1_Acceleration_LeftWrist_Z',
 'Relaxed1_Rotation_LeftWrist_X',
 'Relaxed1_Rotation_LeftWrist_Y',
 'Relaxed1_Rotation_LeftWrist_Z',
 'Relaxed1_Acceleration_RightWrist_X',
 'Relaxed1_Acceleration_RightWrist_Y',
 'Relaxed1_Acceleration_RightWrist_Z',
 'Relaxed1_Rotation_RightWrist_X',
 'Relaxed1_Rotation_RightWrist_Y',
 'Relaxed1_Rotation_RightWrist_Z',
 'Relaxed2_Acceleration_LeftWrist_X',
 'Relaxed2_Acceleration_LeftWrist_Y',
 'Relaxed2_Acceleration_LeftWrist_Z',
 'Relaxed2_Rotation_LeftWrist_X',
 'Relaxed2_Rotation_LeftWrist_Y',
 'Relaxed2_Rotation_LeftWrist_Z',
 'Relaxed2_Acceleration_RightWrist_X',
 'Relaxed2_Acceleration_RightWrist_Y',
 'Relaxed2_Acceleration_RightWrist_Z',
 'Relaxed2_Rotation_RightWrist_X',
 'Relaxed2_Rotation_RightWrist_Y',
 'Relaxed2_Rotation_RightWrist_Z',
 'RelaxedTask1_Acceleration_LeftWrist_X',
 'RelaxedTask1_Acceleration_LeftWrist_Y',
 'RelaxedTask1_Acceleration_LeftWrist_Z'

In [ ]:
time_data_array = np.array(time_data)
time_data_flat = time_data_array.reshape((n_obs,132*976))
time_data_flat.shape

(469, 128832)

In [ ]:
time_data_array.shape
#need to reshape X (it should has 132 features)

(469, 132, 976)

In [ ]:
def initialize_model_lstm(input_shape:tuple, dropout_rate: float = 0.3) -> Model:
    model = Sequential()
    model.add(Input(shape=(input_shape)))
    model.add(layers.Masking(mask_value=0.0))

    #two lstm layers to better capture
    model.add(LSTM(units=64, activation='tanh',
                   return_sequences=True,
                   dropout=dropout_rate,
                   recurrent_dropout=dropout_rate))
    model.add(LSTM(units=32, activation='tanh',
                   dropout=dropout_rate,
                   recurrent_dropout=dropout_rate))

    model.add(layers.Dropout(dropout_rate))

    #dense layer with batch normalization
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(dropout_rate))

    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(dropout_rate))

    #output layer with 3 classification
    model.add(layers.Dense(3, activation='softmax'))

    return model

In [ ]:
model_lstm = initialize_model_lstm(input_shape=X_train.shape[1:])

model_lstm.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy', 'precision', 'recall'])

TypeError: compile() missing required argument 'source' (pos 1)

In [ ]:
es = EarlyStopping(
    monitor="val_loss",
    patience=50,
    restore_best_weights=True,
    verbose=0
)

reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=3,
        verbose=1,
        min_lr=0
    )

history = model_lstm.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    epochs=500,
    batch_size=32,
    callbacks=[es, reduce_lr],
    verbose=1
)
